In [1]:
#Step 1: Load Hugging Face QA model 
from transformers import pipeline

qa_pipeline = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased-distilled-squad')

In [2]:
# !pip install urllib3==1.26 --user
# !pip install requests --upgrade --user
# !pip install selenium --user
import time
import requests
from bs4 import BeautifulSoup

In [3]:
#Step 2: Retrieve and pre-process FAQs from URL 
from selenium import webdriver

browser = webdriver.Chrome()  
browser.get("https://help.fiverr.com/hc/en-us/articles/360011421198-FAQs-for-sellers")

# Wait for the page to load, you can introduce more sophisticated waits here
time.sleep(5)

faq_content = browser.page_source

soup = BeautifulSoup(faq_content, 'html.parser')
faq_paragraphs = soup.find_all('p') 
faqs_unstructured = ' '.join(paragraph.get_text() for paragraph in faq_paragraphs)

browser.quit()

In [4]:
print(faqs_unstructured) 

Find all the answers you need for some of the frequently asked questions for Fiverr Sellers.   Account managementGeneral/miscellaneousPolicy & SafetyPayments and withdrawalsHandling ordersManaging Gigs   1. Is my personal information safe?We care about your privacy. You can read our Privacy Policy here for more information on your personal information.The Privacy Policy is part of our Terms of Service. 2. How do I change/reset my password?You can change or reset your password via your Security settings.   Official Fiverr Username editorial_video 1. Will you ever ask for my password?Customer Support will never ask you for your password, but they may ask you for your security question's answer in specific situations (e.g., to verify account ownership).This is always managed through an official Support request. Here is what you need to know about your password and security: For additional help with policy & safety, please contact Customer Support.     3. Can I partially cancel/refund an o

In [5]:
faqs_unstructured = "1. Is my personal information safe?We care about your privacy. You can read our Privacy Policy here for more information on your personal information.The Privacy Policy is part of our Terms of Service. 2. How do I change/reset my password?You can change or reset your password via your Security settings.   Official Fiverr Username editorial_video 1. Will you ever ask for my password?Customer Support will never ask you for your password, but they may ask you for your security question's answer in specific situations (e.g., to verify account ownership).This is always managed through an official Support request. Here is what you need to know about your password and security: For additional help with policy & safety, please contact Customer Support.     3. Can I partially cancel/refund an order?Orders can not be partially canceled or refunded at this time.Once an order is created and there is a need to partially cancel/refund your buyer, we recommend that you ask your buyer to place a new order for the newly agreed-upon price. Once that new order is created and then marked as Complete, you can cancel the original order.   4. Can I accept a tip?You can accept a tip, however, it's advised not to ask for one.Do your best as a seller—ultimately it's up to the buyer whether to leave a tip or not. Learn more with our Communication Webinar.   1. What are Packages?Packages are a structure for selling services on Fiverr that allow you to combine your basic Gig with built-in revisions and Extras to create an offer that gives Buyers exactly what they want. ammyy.com Note: General websites (such as google.com), or websites that provide free tools (such as wordcount.com or colorpicker.com) are allowed. If you use a URL that is not listed above, your Gig may be denied.Repetitive violations may also cause an account to be restricted. "

In [6]:
import re

def extract_faqs(faq_text):
    # Split using pattern of numbers followed by a dot
    parts = re.split(r'(\d+\.)', faq_text)[1:]

    # Group by twos (as we've split by the numbers, we want to rejoin them)
    grouped_parts = ["".join(parts[i:i+2]) for i in range(0, len(parts), 2)]

    faqs = []

    for part in grouped_parts:
        # Splitting by the first question mark to separate the question and the answer
        q, a = part.split("?", 1)  
        faqs.append({
            'question': q.strip() + '?',
            'answer': a.strip()
        })

    return faqs

faqs = extract_faqs(faqs_unstructured)

for faq in faqs:
    print("Q:", faq['question'])
    print("A:", faq['answer'])
    print("-" * 40)


Q: 1. Is my personal information safe?
A: We care about your privacy. You can read our Privacy Policy here for more information on your personal information.The Privacy Policy is part of our Terms of Service.
----------------------------------------
Q: 2. How do I change/reset my password?
A: You can change or reset your password via your Security settings.   Official Fiverr Username editorial_video
----------------------------------------
Q: 1. Will you ever ask for my password?
A: Customer Support will never ask you for your password, but they may ask you for your security question's answer in specific situations (e.g., to verify account ownership).This is always managed through an official Support request. Here is what you need to know about your password and security: For additional help with policy & safety, please contact Customer Support.
----------------------------------------
Q: 3. Can I partially cancel/refund an order?
A: Orders can not be partially canceled or refunded at 

In [7]:
print(faqs)

[{'question': '1. Is my personal information safe?', 'answer': 'We care about your privacy. You can read our Privacy Policy here for more information on your personal information.The Privacy Policy is part of our Terms of Service.'}, {'question': '2. How do I change/reset my password?', 'answer': 'You can change or reset your password via your Security settings.   Official Fiverr Username editorial_video'}, {'question': '1. Will you ever ask for my password?', 'answer': "Customer Support will never ask you for your password, but they may ask you for your security question's answer in specific situations (e.g., to verify account ownership).This is always managed through an official Support request. Here is what you need to know about your password and security: For additional help with policy & safety, please contact Customer Support."}, {'question': '3. Can I partially cancel/refund an order?', 'answer': 'Orders can not be partially canceled or refunded at this time.Once an order is cr

In [8]:
#Step 4: running the model

def get_answer_from_faq(user_question, structured_faqs):
    relevant_contexts = []
    
    for faq in structured_faqs:
        if any(word.lower() in (faq['question'] + faq['answer']).lower() for word in user_question.split()):
            relevant_contexts.append(faq['answer'])
    
    combined_context = ' '.join(relevant_contexts)
    
    # Handle empty context:
    if not combined_context.strip():
        # Option a:
        return {'answer': "Sorry, I couldn't find an answer to that."}

        # Option b (uncomment to use):
        # combined_context = ' '.join([faq['answer'] for faq in structured_faqs])
    
    return qa_pipeline({
        'context': combined_context,
        'question': user_question
    })

# Example usage:
question = "How do I reset my password?"
answer = get_answer_from_faq(question, faqs)
print(answer['answer'])

via your Security settings


In [ ]:
# #Step 4: Running the model
# def get_answer_from_faq(question, faq_content):
#     return qa_pipeline({
#         'context': faq_content,
#         'question': question
#     })

In [9]:
#Testing
question = "How can I contact your support team?"
answer = get_answer_from_faq(question, faqs)
print(answer['answer'])

Communication Webinar


In [10]:
#Testing
question = "Can I cancel my withdrawal?"
answer = get_answer_from_faq(question, faqs)
print(answer['answer'])

If you use a URL that is not listed above


In [11]:
#Testing
question = "Do I have to pay taxes from my Fiverr earnings?"
answer = get_answer_from_faq(question, faqs)
print(answer['answer'])

Learn more with our Communication Webinar


In [12]:
#Testing
question = "How can I increase sales?"
answer = get_answer_from_faq(question, faqs)
print(answer['answer'])

combine your basic Gig with built-in revisions and Extras


In [13]:
#Testing
question = "Is my personal information safe?"
answer = get_answer_from_faq(question, faqs)
print(answer['answer'])

Learn more with our Communication Webinar
